In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress
from pathlib import Path
import tomllib

In [8]:
def get_resistance(voltage: np.ndarray, current: np.ndarray) -> np.ndarray:
    return voltage / current

## Boltzmann-data

In [9]:
BOLTZMANN_DIR = Path(__name__).parent

### $R_0$

In [10]:
def get_wheatstone_resistance(R1: float, R2: float, R3: float) -> float:
    return R3*R2/R1

In [11]:
with open(BOLTZMANN_DIR / "wheatstone.toml", "rb") as f:
    wheatstone_data = tomllib.load(f)

R0 = get_wheatstone_resistance(
    wheatstone_data["R1"],
    wheatstone_data["R2"] - wheatstone_data["Kabel"],
    wheatstone_data["R3"],
)

### Motstand i pæren

In [12]:
data_boltzmann = np.loadtxt(
    BOLTZMANN_DIR / "results.csv", delimiter=",", skiprows=1, unpack=True
)
boltzmann_resistance = np.array(
    [
        get_resistance(data_boltzmann[0], data_boltzmann[2]),  # Multimeter
        get_resistance(data_boltzmann[1], data_boltzmann[3]),  # Strømforsyning
    ]
)
relative_resistance = boltzmann_resistance / R0
np.savetxt(
    BOLTZMANN_DIR / "relative_resistance.csv",
    relative_resistance,
    fmt="%.2f",
    delimiter=",",
)

### Temperatur

In [13]:
def interpolate_temperature(
    resistance: float,
    resistance_interval: tuple[float, float],
    temperature_interval: tuple[float, float],
) -> float:
    delta_R = resistance_interval[1] - resistance_interval[0]
    delta_T = temperature_interval[1] - temperature_interval[0]
    return resistance_interval[0] + delta_T / delta_R * (
        resistance - resistance_interval[0]
    )


def get_resistance_interval(
    resistance: float, resistance_array: np.ndarray
) -> tuple[int, int]:
    for i in resistance_array:
        if i <= resistance:
            return i, i + 1
    return 0, 1

### Multimeter

In [14]:
reference_resistance, temperature, _ = np.loadtxt(
    BOLTZMANN_DIR / "resistance_temperature_table.csv", delimiter=",", skiprows=1, unpack=True
)

temperature_results = np.zeros_like(relative_resistance[0])
for index, resistance in enumerate(relative_resistance[0]):
    lower, upper = get_resistance_interval(resistance, reference_resistance)
    print(lower, upper)
    temperature_results[index] = interpolate_temperature(
        resistance,
        (reference_resistance[lower], reference_resistance[upper]),
        (temperature[lower], temperature[upper]),
    )
print(temperature)

1.0 2.0


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

## Leslie-data

In [ ]:
LESLIE_DIR = Path(__name__).parent / "Leslie"

In [ ]:
data_leslie = pd.read_csv(LESLIE_DIR / "results.csv")
